In [1]:
from __future__ import print_function
import sys
sys.path.append('../build/')
%pylab inline
np.set_printoptions(precision=4, suppress=True)
import versor as vsr
import hyperdual
import motor_jacobian

Populating the interactive namespace from numpy and matplotlib


In [2]:
rotor = (vsr.Biv(1,0,0).unit() * pi/6).exp()
rotor1 = vsr.Rot(1,0,0,0)
a = vsr.Vec(1,0,0)
b = a.spin(rotor)

In [3]:
# hyperdual.hyperdual1(a,b,rotor)

In [4]:
# hyperdual.hyperdual2(a,b,rotor)

In [5]:
 def jacrot(a, rot):
    j = np.zeros((3,3))
    for i in range(3):
        biv = vsr.Biv(0.0, 0.0, 0.0)
        biv[i] = 1.0
        j[:,i] = 2.0 * np.array(biv.comm(a.spin(rot))) # Wrong sign?
    return j

In [48]:
J = jacrot(a,rotor1)
print(J)

[[ 0.  2.  0.]
 [ 0.  0.  2.]
 [ 0.  0.  0.]]


In [51]:
np.array([jacrot(a,rot) for a in vectors_a]).reshape(-1,3)

array([[ 0.406 , -0.2892,  0.    ],
       [-0.4841,  0.    , -0.2892],
       [ 0.    , -0.4841, -0.406 ],
       [ 1.9212,  0.0154,  0.    ],
       [-0.2697,  0.    ,  0.0154],
       [ 0.    , -0.2697, -1.9212],
       [ 0.4031,  0.1259,  0.    ],
       [ 3.08  ,  0.    ,  0.1259],
       [ 0.    ,  3.08  , -0.4031],
       [-3.0692,  0.1342,  0.    ],
       [ 1.0462,  0.    ,  0.1342],
       [ 0.    ,  1.0462,  3.0692],
       [ 1.3915, -0.9433,  0.    ],
       [ 0.892 ,  0.    , -0.9433],
       [ 0.    ,  0.892 , -1.3915],
       [-0.3187,  2.0026,  0.    ],
       [-1.6942,  0.    ,  2.0026],
       [ 0.    , -1.6942,  0.3187],
       [-2.087 , -0.9905,  0.    ],
       [-1.029 ,  0.    , -0.9905],
       [ 0.    , -1.029 ,  2.087 ],
       [-2.1767, -1.2374,  0.    ],
       [ 2.0351,  0.    , -1.2374],
       [ 0.    ,  2.0351,  2.1767],
       [-0.8041, -0.9026,  0.    ],
       [ 1.8108,  0.    , -0.9026],
       [ 0.    ,  1.8108,  0.8041],
       [-3.3761,  2.4591,  0

In [7]:
a ^ b * 2

Biv: [ 1.732 0 0 ]

In [8]:
res, grad, H = hyperdual.hyperdualB(a, b, rotor1) 
print(grad)
print()
print(H)

[ 1.7321  0.      0.    ]

[[ 3.      0.      0.    ]
 [ 0.      3.      1.7321]
 [ 0.      1.7321  1.    ]]


In [44]:
vectors_a = [vsr.Vec(1,0,0), vsr.Vec(0,1,0), vsr.Vec(0,0,1)]
vectors_b = [a.spin(rotor) for a in vectors_a]

In [45]:
n_points=10
vectors_a = [vsr.Vec(*np.random.normal(0.0, 0.8, 3)) for i in range(n_points)]
vectors_b = [vector.spin(rotor) for vector in vectors_a]

In [46]:
def update(rot):
    ans = [hyperdual.hyperdualB(a,b,rot) for a,b in zip(vectors_a, vectors_b)]
    res = np.sum([t[0] for t in ans])
    grad = np.sum([t[1] for t in ans], axis=0)
    Hess = np.sum([t[2] for t in ans], axis=0)
    B = np.dot(np.linalg.pinv(Hess),grad)
    rot = vsr.Biv(*B).exp() * rot
    grad_norm = np.linalg.norm(grad)
    return rot, res, grad_norm

In [47]:
rot = vsr.Rot(1,0,0,0)
for i in range(3):
    rot, res, grad_norm = update(rot)
    print(res)
print(rot)

7.63933287844
0.0882019528161
8.20768249977e-08
Rot: [ 0.87 -0.5 3.2e-17 -2.6e-17 ]


In [58]:
rot = vsr.Rot(1,0,0,0)
J = np.array([jacrot(a,rot) for a in vectors_a]).reshape(-1,3)
print(np.dot(J.T,J))

[[ 61.1147  -5.8339   2.6544]
 [ -5.8339  55.5389   0.8801]
 [  2.6544   0.8801  34.3606]]


In [59]:
Hess = np.sum([hyperdual.hyperdualB(a,b,rot)[2] for a,b in zip(vectors_a, vectors_b)], axis=0)
print(Hess)

[[ 45.836   -5.5248  -0.5354]
 [ -5.5248  49.4821   9.6105]
 [ -0.5354   9.6105  40.4173]]
